In [ ]:
''' To identify polygons that are not contained within other shapes in the same layer using ArcPy, you can use the arcpy library's geometry comparison methods, 
such as within or contains. '''

import arcpy, pandas as pd

# Input feature class
root_folder = "Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Sts_Drainage_Areas\GRDC_Watersheds"
input_fc = root_folder + "\CSS-WATERSHEDS-MERGE_FINAL_SELECTION.shp"
# Pre-load all polygons into memory for faster access
all_polygons = []
with arcpy.da.SearchCursor(input_fc, ["grdcno_int", "SHAPE@"]) as cursor:
    for row in cursor:
        all_polygons.append(row)

print(f"Loaded {len(all_polygons)} polygons")

Loaded 809 polygons


In [13]:
# Store grdcno_int values of polygons that do not contain any other polygon
non_containing = []
# Track polygons that have been found to contain others (to skip in future iterations)
containing_grdcnos = set()
# Track which polygons we've already fully processed
processed_grdcnos = set()

# Iterate through each polygon
for i, (grdcno1, geom1) in enumerate(all_polygons):
    # Skip if this polygon is already known to contain others
    if grdcno1 in containing_grdcnos:
        continue
    
    contains_others = False
    
    # Compare with all other polygons
    for j, (grdcno2, geom2) in enumerate(all_polygons):
        # Skip self-comparison
        if grdcno1 == grdcno2:
            continue
            
        # Skip if we've already determined grdcno2 contains others
        if grdcno2 in containing_grdcnos:
            continue
        
        # Check if the current polygon contains another polygon
        if geom2.within(geom1):
            contains_others = True
            containing_grdcnos.add(grdcno1)  # Mark this polygon as containing others
            break
    
    # If doesn't contain others, add to the list
    if not contains_others:
        non_containing.append(grdcno1)
        
    # Mark as processed
    processed_grdcnos.add(grdcno1)
    
    # Progress indicator
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{len(all_polygons)} polygons")

# Output the result
non_containing = sorted(non_containing)
containing_grdcnos = sorted(containing_grdcnos)

pd.DataFrame({'grdcno_int': non_containing}).to_csv(f"{root_folder}/non_containing_grdcno_int.csv", index_label="Index")

print(f"\nResults:")
print(f"Polygons that do NOT contain any other polygon (grdcno_int): {len(non_containing)}\n\t{non_containing}")
print(f"Polygons found to contain others: {len(containing_grdcnos)}\n\t{containing_grdcnos}")
print(f"Total polygons processed: {len(all_polygons)}")

Processed 100/809 polygons
Processed 200/809 polygons
Processed 300/809 polygons
Processed 400/809 polygons
Processed 500/809 polygons
Processed 600/809 polygons
Processed 700/809 polygons
Processed 800/809 polygons

Results:
Polygons that do NOT contain any other polygon (grdcno_int): 611
	[3617814, 3618700, 3618722, 3618950, 3621200, 3628400, 3628701, 3628900, 3630600, 3631100, 3632400, 3633180, 3633301, 3634150, 3634370, 3635301, 3635310, 3635360, 3635430, 3635440, 3635451, 3635600, 3637380, 3637775, 3637810, 3637910, 3638700, 3649010, 3649030, 3649050, 3649110, 3649130, 3649151, 3649160, 3649210, 3649240, 3649310, 3649311, 3649321, 3649325, 3649420, 3649423, 3649440, 3649450, 3649455, 3649460, 3649511, 3649610, 3649850, 3649855, 3649960, 4101900, 4102100, 4103630, 4103650, 4103750, 4105820, 4105850, 4113310, 4113315, 4113351, 4113600, 4115100, 4115210, 4115224, 4115225, 4115230, 4115235, 4115241, 4115310, 4115320, 4115330, 4115345, 4115351, 4115410, 4116150, 4116201, 4116311, 41163